# Chapter 2. Ingesting Data into the Cloud

* 싸이그래머 / CloudΨ - DS on GCP [1]
* 김무성

# 차례 
* 준비단계    
* Airline On-Time Perfomance Data
    * Knowability
    * Training–Serving Skew
    * Download Procedure
    * Dataset Attributes
* Why Not Store the Data in Situ?
    * Scaling Up
    * Scaling Out
    * Data in Situ with Colossus and Jupiter
* Ingesting Data
    * Reverse Engineering a Web Form
    * Dataset Download
    * Exploration and Cleanup
    * Uploading Data to Google Cloud Storage
* Scheduling Monthly Downloads
    * Ingesting in Python
    * Flask Web App
    * Running on App Engine
    * Securing the URL
    * Scheduling a Cron Task
* Summary
* Code Break

----------------------------

#### 참고 
* Getting started with Google Cloud Training Material - 2018 - https://www.slideshare.net/jkbaseer/getting-started-with-google-cloud-training-material-2018

<img src="https://image.slidesharecdn.com/cloudonboardtrainingmanual2018sg17april2018-180417132701/95/getting-started-with-google-cloud-training-material-2018-17-1024.jpg?cb=1523971737" width=800 />
<img src="https://image.slidesharecdn.com/cloudonboardtrainingmanual2018sg17april2018-180417132701/95/getting-started-with-google-cloud-training-material-2018-18-1024.jpg?cb=1523971737" width=800 />
<img src="https://image.slidesharecdn.com/cloudonboardtrainingmanual2018sg17april2018-180417132701/95/getting-started-with-google-cloud-training-material-2018-20-1024.jpg?cb=1523971737" width=800 />

# 준비단계

----------------------------

# Airline On-Time Perfomance Data
* Knowability
* Training–Serving Skew
* Download Procedure
* Dataset Attributes

<img src="figures/cap01.png" width=600 />

## Knowability

## Training–Serving Skew

## Download Procedure

<img src="figures/cap02.png" width=600 />

## Dataset Attributes

----------------------------

# Why Not Store the Data in Situ?
* Scaling Up
* Scaling Out
* Data in Situ with Colossus and Jupiter

<img src="figures/cap03.png" width=600 />

## Scaling Up

<img src="figures/cap04.png" width=600 />

## Scaling Out

<img src="figures/cap05.png" width=600 />

## Data in Situ with Colossus and Jupiter

<img src="figures/cap06.png" width=600 />

-----------------------

# Ingesting Data
* Reverse Engineering a Web Form
* Dataset Download
* Exploration and Cleanup
* Uploading Data to Google Cloud Storage   

<img src="figures/cap07.png" width=600 />

## Reverse Engineering a Web Form

<img src="figures/cap08.png" width=600 />

<img src="figures/cap09.png" width=600 />

<img src="figures/cap10.png" width=600 />

<img src="figures/cap11.png" width=600 />

## Dataset Download

<img src="figures/cap12.png" width=600 />

<img src="figures/cap13.png" width=600 />

## Exploration and Cleanup

## Uploading Data to Google Cloud Storage

-----------------------

# Scheduling Monthly Downloads
* Ingesting in Python
* Flask Web App
* Running on App Engine
* Securing the URL
* Scheduling a Cron Task

<img src="figures/cap14.png" width=600 />

## Ingesting in Python

## Flask Web App

## Running on App Engine

## Securing the URL

## Scheduling a Cron Task

-----------------------

# Summary

-----------------------

# Code Break

# 참고자료
* [1] Data Science on the Google Cloud Platform: Implementing End-to-End Real-Time Data Pipelines: From Ingest to Machine Learning - https://www.amazon.com/Data-Science-Google-Cloud-Platform/dp/1491974567
* [2] Book github - https://github.com/GoogleCloudPlatform/data-science-on-gcp